# Stream Order vs Threshold
At each threshold, take the  Stream_Order across all basins (integer).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip -q install rasterio whitebox geopandas

In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio
import networkx as nx
import matplotlib.pyplot as plt
import whitebox
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

## Config

In [ ]:
DRIVE_ROOT = '/content/drive/My Drive'
PROJECT_DIR = os.path.join(DRIVE_ROOT, 'Oyster_Flow')
RESULT_DIR = os.path.join(PROJECT_DIR, 'OFG_result')
os.makedirs(RESULT_DIR, exist_ok=True)

flow_accum = os.path.join(PROJECT_DIR, 'FlowAcc_Flow51.tif')
flow_dir = os.path.join(PROJECT_DIR, 'FlowDir_Fill31.tif')

streams_rast = os.path.join(RESULT_DIR, 'HS_slope_streams.tif')
stream_order_r = os.path.join(RESULT_DIR, 'HS_slope_order.tif')

basin_shp = os.path.join(PROJECT_DIR, 'OFG_New', 'oysterbasin0325.shp')

THRESHOLDS = list(range(35))

wbt = whitebox.WhiteboxTools()
wbt.set_working_dir(RESULT_DIR)

## Helpers

In [ ]:
def extract_graph(threshold):
    wbt.extract_streams(flow_accum, streams_rast, threshold)
    wbt.strahler_stream_order(d8_pntr=flow_dir, streams=streams_rast, output=stream_order_r)

    with rasterio.open(streams_rast) as src:
        streams_data = src.read(1).astype(np.float32)
        transform = src.transform
        streams_crs = src.crs
    with rasterio.open(flow_accum) as src:
        flow_acc_data = src.read(1).astype(np.float32)
    with rasterio.open(stream_order_r) as src:
        stream_order_data = src.read(1).astype(np.float32)

    flow_acc_data[flow_acc_data < 0] = 0

    graph = nx.Graph()
    rows, cols = np.where(streams_data > 0)
    for r, c in zip(rows, cols):
        graph.add_node((r, c), flow_acc=flow_acc_data[r, c], stream_order=stream_order_data[r, c])
    for r, c in zip(rows, cols):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < streams_data.shape[0] and 0 <= nc < streams_data.shape[1]:
                if streams_data[nr, nc] > 0:
                    graph.add_edge((r, c), (nr, nc))
    return graph, transform, streams_crs

def assign_basins(graph, transform, streams_crs, basins):
    node_list = []
    for node in graph.nodes():
        r, c = node
        lon, lat = rasterio.transform.xy(transform, r, c)
        node_list.append({'node': node, 'lon': lon, 'lat': lat})
    node_df = pd.DataFrame(node_list)
    node_gdf = gpd.GeoDataFrame(
        node_df,
        geometry=[Point(xy) for xy in zip(node_df['lon'], node_df['lat'])],
        crs=streams_crs
    )
    node_basins = gpd.sjoin(node_gdf, basins, how='left', predicate='within')
    if 'basin_id' not in node_basins.columns:
        node_basins['basin_id'] = node_basins.index_right
    node_basins = node_basins.dropna(subset=['basin_id'])
    node_basins['basin_id'] = node_basins['basin_id'].astype(int)
    return node_basins

def basin_max_stream_order(graph, node_basins):
    max_vals = []
    for bid in node_basins['basin_id'].unique():
        subset = node_basins[node_basins['basin_id'] == bid]
        nodes_in_basin = subset['node'].tolist()
        subgraph = graph.subgraph(nodes_in_basin)
        if len(subgraph) < 1:
            continue
        max_order = int(np.max([d['stream_order'] for _, d in subgraph.nodes(data=True)]))
        max_vals.append(max_order)
    return max_vals


In [ ]:
# Env vars plot (th=0)
base_dir = os.path.join(PROJECT_DIR, 'OFG_New')
env_vars = ['w1', 'v1', 'temp1', 'NO32', 'chlorophyll1', 'salt1']
display_names = {'NO32': 'NO$_3$2'}

def load_env_arrays(var_list, base_directory):
    arrays = {}
    for v in var_list:
        path = os.path.join(base_directory, f'{v}.tif')
        with rasterio.open(path) as src:
            arr = src.read(1).astype(np.float32)
            if src.nodata is not None:
                arr = np.where(arr == src.nodata, np.nan, arr)
        arrays[v] = arr
    return arrays

def binned_curve(df, x_col, y_col, n_bins=20):
    x = df[x_col].values
    y = df[y_col].values
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]
    if x.size == 0:
        return np.array([]), np.array([])
    bins = np.linspace(np.min(x), np.max(x), n_bins + 1)
    inds = np.digitize(x, bins)
    centers = []
    means = []
    for i in range(1, len(bins)):
        sel = y[inds == i]
        if sel.size > 0:
            centers.append((bins[i-1] + bins[i]) / 2)
            means.append(np.mean(sel))
    return np.array(centers), np.array(means)

env_arrays = load_env_arrays(env_vars, base_dir)
basins = gpd.read_file(basin_shp)

# th=0
graph, transform, streams_crs = extract_graph(0)
node_basins = assign_basins(graph, transform, streams_crs, basins)

# compute stream order per basin
rows = []
for bid in node_basins['basin_id'].unique():
    sub = node_basins[node_basins['basin_id'] == bid]
    nodes_in_basin = sub['node'].tolist()
    subgraph = graph.subgraph(nodes_in_basin)
    if len(subgraph) < 1:
        continue
    max_order = int(np.max([d['stream_order'] for _, d in subgraph.nodes(data=True)]))
    rows.append({'basin_id': bid, 'stream_order': max_order})

bdf = pd.DataFrame(rows)

# attach basin mean env values
env_rows = []
for bid in bdf['basin_id'].unique():
    sub = node_basins[node_basins['basin_id'] == bid]
    vals = {'basin_id': bid}
    for v, arr in env_arrays.items():
        vs = []
        for node in sub['node']:
            r, c = node
            try:
                vs.append(float(arr[r, c]))
            except Exception:
                pass
        vals[v] = float(np.nanmean(vs)) if vs else np.nan
    env_rows.append(vals)
env_df = pd.DataFrame(env_rows)
bdf = bdf.merge(env_df, on='basin_id', how='left')

fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharey=True)
axes = axes.flatten()
for i, var in enumerate(env_vars):
    ax = axes[i]
    x, y = binned_curve(bdf, var, 'stream_order', n_bins=20)
    ax.plot(x, y, marker='o', markersize=3)
    ax.set_title(display_names.get(var, var))
    ax.set_xlabel(display_names.get(var, var))
    if i % 2 == 0:
        ax.set_ylabel('Stream_Order')
    ax.grid(True)

plt.suptitle('Stream_Order vs env vars (th=0)')
plt.tight_layout()
out_pdf = os.path.join(RESULT_DIR, 'stream_order_vs_env_th_0.pdf')
plt.savefig(out_pdf, bbox_inches='tight')
plt.show()
